<a href="https://colab.research.google.com/github/Emmanuelara1/Life_Analytics/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, jsonify
import os
from werkzeug.utils import secure_filename
from datetime import datetime, timedelta
import jwt
from bio_image_analysis import process_image, save_result, display_image
from functools import wraps

In [ ]:
SECRET_KEY = 'your_jwt_secret_key'
EXPIRATION_TIME = timedelta(hours=1)

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static/uploads/'
app.config['ALLOWED_EXTENSIONS'] = {'png', 'jpg', 'jpeg', 'gif'}

# Simulate a user database (you can integrate with a real database)
users = {"admin": {"password": "adminpassword", "email": "admin@example.com"}}

# JWT authentication decorator
def token_required(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        token = request.args.get('token')  # Retrieve token from URL args
        if not token:
            return jsonify({'message': 'Token is missing!'}), 403
        try:
            data = jwt.decode(token, SECRET_KEY, algorithms=["HS256"])
            current_user = users.get(data['username'])
        except Exception as e:
            return jsonify({'message': 'Token is invalid!'}), 403
        return f(current_user, *args, **kwargs)
    return decorated_function

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

@app.route('/login', methods=['POST'])
def login():
    auth = request.json
    username = auth.get('username')
    password = auth.get('password')

    if username in users and users[username]['password'] == password:
        # Generate JWT token
        token = jwt.encode({'username': username, 'exp': datetime.utcnow() + EXPIRATION_TIME}, SECRET_KEY, algorithm="HS256")
        return jsonify({'token': token})
    else:
        return jsonify({'message': 'Invalid credentials'}), 401

@app.route('/upload', methods=['POST'])
@token_required
def upload_file(current_user):
    file = request.files['file']
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        # Process the image and classify
        preprocessed_image, segmented_image, label, prediction = process_image(filepath)

        # Save processed images
        processed_image_path = os.path.join(app.config['UPLOAD_FOLDER'], 'processed_' + filename)
        segmented_image_path = os.path.join(app.config['UPLOAD_FOLDER'], 'segmented_' + filename)
        save_result(preprocessed_image, processed_image_path)
        save_result(segmented_image, segmented_image_path)

        return jsonify({
            'processed_image': processed_image_path,
            'segmented_image': segmented_image_path,
            'label': label,
            'prediction': prediction.tolist()
        })
    return jsonify({'message': 'Invalid file type'}), 400

if __name__ == '__main__':
    app.run(debug=True)
